In [7]:
import pyro
import dill
import os
from pathlib import Path
import pandas as pd
import numpy as np

from pyciemss.PetriNetODE.interfaces import load_petri_model
from pyciemss.Ensemble.interfaces import setup_model, sample, calibrate
from pyciemss.utils.interface_utils import convert_to_output_format

In [8]:
DATA_PATH = Path(".")/"covid_data"
data_filename = DATA_PATH / "US_case_hospital_death.csv"
data = pd.read_csv(data_filename)

# Clip off the first 2 months of data
data_total_population = 331893745
train_start_date = "2021-12-01"
test_start_date = "2022-03-01"
test_end_date = "2022-04-01"


In [21]:
def get_train_test_data(data: pd.DataFrame, train_start_date: str, test_start_date: str, test_end_date: str) -> pd.DataFrame:
    train_df = data[(data['date'] >= train_start_date) & (data['date'] < test_start_date)]
    train_data = [0] * train_df.shape[0]
    start_time = train_df.index[0]

    train_cases = np.array(train_df["case_census"].astype("float")) / data_total_population
    train_timepoints = np.array(train_df.index.astype("float"))

    test_df = data[(data['date'] >= test_start_date) & (data['date'] < test_end_date)]
    test_cases = np.array(test_df["case_census"].astype("float")) / data_total_population
    test_timepoints = np.array(test_df.index.astype("float"))

    for time, row in train_df.iterrows():
        row_dict = {}
        row_dict["Cases"] = row["case_census"] / data_total_population
        row_dict["Dead"] = row["cumulative_deaths"] / data_total_population
        if row["hospital_census"] > 0:
            row_dict["Hospitalizations"] = row["hospital_census"] / data_total_population

        index = time - start_time
        train_data[index] = (float(time), row_dict)
    
    all_timepoints = np.concatenate((train_timepoints, test_timepoints))

    return train_data, train_cases, train_timepoints, test_cases, test_timepoints, all_timepoints

train_data, train_cases, train_timepoints, test_cases, test_timepoints, all_timepoints = get_train_test_data(data, train_start_date, test_start_date, test_end_date)

### Setup Models

In [9]:
MIRA_PATH = Path("..")/".."/"test"/"models"/"april_ensemble_demo"

In [16]:
filename1 = MIRA_PATH/"BIOMD0000000955_template_model.json"
model1 = load_petri_model(str(filename1), add_uncertainty=True)

def create_start_state1(data, start_date):
    start_state = data.set_index('date').loc[start_date].to_dict()

    returned_state = {}
    returned_state["Extinct"] = start_state['cumulative_deaths']
    
    if np.isnan(start_state['hospital_census']):
        returned_state["Threatened"] = start_state['case_census'] / 100
    else:
        returned_state["Threatened"] = start_state['hospital_census']

    returned_state["Diagnosed"] = start_state['case_census'] / 2
    returned_state["Recognized"] = start_state['case_census'] / 2
    returned_state['Infected'] = start_state['case_census'] * 5
    returned_state['Ailing'] = returned_state["Threatened"] * 50

    returned_state['Healed'] = start_state['cumulative_deaths'] * 50
    returned_state['Susceptible'] =  data_total_population - sum(returned_state.values())
    assert(returned_state['Susceptible'] > 0)
    return {k:v/data_total_population for k, v in returned_state.items()}

start_state1 = create_start_state1(data, train_start_date)

def solution_mapping1(model1_solution: dict) -> dict:
    mapped_dict = {}
    mapped_dict["Cases"] = model1_solution["Diagnosed"] + model1_solution["Recognized"]
    mapped_dict["Hospitalizations"] = model1_solution["Threatened"]
    mapped_dict["Dead"] = model1_solution["Extinct"]
    return mapped_dict

In [17]:
filename2 = MIRA_PATH/"BIOMD0000000960_template_model.json"
model2 = load_petri_model(str(filename2), add_uncertainty=True)

mira_start_state2 = {k[0]: v.data['initial_value'] for k, v in model2.G.variables.items()}
model2_total_population = sum(mira_start_state2.values())

def create_start_state2(data, start_date):
    start_state = data.set_index('date').loc[start_date].to_dict()

    returned_state = {}
    returned_state["Deceased"] = start_state['cumulative_deaths']
    
    if np.isnan(start_state['hospital_census']):
        returned_state["Hospitalized"] = start_state['case_census'] / 100
    else:
        returned_state["Hospitalized"] = start_state['hospital_census']
    
    returned_state["Infectious"] = start_state['case_census']

    returned_state['Exposed'] = start_state['case_census'] * 10
    returned_state['Asymptomatic'] = start_state['case_census'] * 5

    returned_state['Recovered'] = start_state['cumulative_deaths'] * 10

    returned_state['Susceptible'] =  data_total_population - sum(returned_state.values())
    assert(returned_state['Susceptible'] > 0)
    return {k:v*(model2_total_population/data_total_population) for k, v in returned_state.items()}

start_state2 = create_start_state2(data, train_start_date)

def solution_mapping2(model2_solution: dict) -> dict:
    mapped_dict = {}
    mapped_dict["Cases"] = model2_solution["Infectious"] / model2_total_population
    mapped_dict["Hospitalizations"] = model2_solution["Hospitalized"] / model2_total_population
    mapped_dict["Dead"] = model2_solution["Deceased"] / model2_total_population
    return mapped_dict

/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('Susceptible', ('identity', 'ido:0000514')), ('Exposed', ('identity', 'apollosv:00000154')), ('Infectious', ('identity', 'ido:0000511'), ('disease_severity', 'ncit:C25269'), ('hospitalization', 'ncit:C68851')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1). This is likely to be an error.
  warnings.warn(warnings_string)
/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('Susceptible', ('identity', 'ido:0000514')), ('Exposed', ('identity', 'apollosv:00000154')), ('Asymptomatic', ('identity', 'ido:0000511'), ('disease_severity', 'ncit:C3833'), ('hospitalization', 'ncit:C68851')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1). This is likely to be an error.
  warnings.warn(warnings_string)
/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWa

In [18]:
filename3 = MIRA_PATH/"BIOMD0000000983_template_model.json"
model3 = load_petri_model(str(filename3), add_uncertainty=True)

mira_start_state3 = {k[0]: v.data['initial_value'] for k, v in model3.G.variables.items()}
mira_start_state3['Deceased'] = 0.0

model3_total_population = sum(mira_start_state3.values())
h = 0.01

def create_start_state3(data, start_date):
    start_state = data.set_index('date').loc[start_date].to_dict()

    returned_state = {}

    if np.isnan(start_state['hospital_census']):
        returned_state["Infected_unreported"] = start_state['case_census'] / (100 * h)
    else:
        returned_state["Infected_unreported"] = start_state['hospital_census']/h

    returned_state["Infected_reported"] = start_state['case_census']
    returned_state["Deceased"] = start_state['cumulative_deaths']
    returned_state['Exposed'] = start_state['case_census'] * 10
    returned_state['Recovered'] = start_state['cumulative_deaths'] * 5
    returned_state['Quarantined'] = returned_state["Infected_reported"] * 0.5

    returned_state['Susceptible_unconfined'] =  data_total_population - sum(returned_state.values())
    assert(returned_state['Susceptible_unconfined'] > 0)
    return {k:v*(model3_total_population/data_total_population) for k, v in returned_state.items()}


start_state3 = create_start_state3(data, train_start_date)

def solution_mapping3(model3_solution: dict) -> dict:
    mapped_dict = {}
    # Ideally we would make better use of this distinction between "reported" and "unreported".
    # However, as other models don't include this distinction, we must instead sum them together to make solution outputs comparable.
    mapped_dict["Cases"] = model3_solution["Infected_reported"] / model3_total_population
    mapped_dict["Hospitalizations"] = (model3_solution["Infected_unreported"] * h) / model3_total_population
    mapped_dict["Dead"] = (model3_solution["Deceased"]) / model3_total_population
    return mapped_dict



/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('Susceptible_unconfined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C68851')), ('Exposed', ('identity', 'apollosv:00000154')), ('Infected_reported', ('identity', 'ido:0000511'), ('diagnosis', 'ncit:C15220')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1). This is likely to be an error.
  warnings.warn(warnings_string)
/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/base.py:342: UserWarning: Parameter (('Susceptible_unconfined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C68851')), ('Exposed', ('identity', 'apollosv:00000154')), ('Infected_unreported', ('identity', 'ido:0000511'), ('diagnosed', 'ncit:C113725')), 'ControlledConversion', 'rate') has value None and will be set to Uniform(0, 1). This is likely to be an error.
  warnings.warn(warnings_string)
/Users/cott741/projects/ASKEM/pyciemss/src/pyciemss/PetriNetODE/bas

### Setup Ensemble

In [22]:
models = [model1, model2, model3]
weights = [1/3, 1/3, 1/3]
start_time = train_timepoints[0] - 1e-5

start_states = [start_state1, start_state2, start_state3]
solution_mappings = [solution_mapping1, solution_mapping2, solution_mapping3]

ensemble_total_population = 1.0
dirichlet_concentration = 1.0

ensemble = setup_model(models, 
                       weights, 
                       solution_mappings, 
                       start_time, 
                       start_states, 
                       dirichlet_concentration=dirichlet_concentration)
ensemble

Ensemble of 3 models. 

 	Dirichlet Alpha: tensor([0.3333, 0.3333, 0.3333]). 

 	Models: [ScaledNormalNoisePetriNetODESystem(
	beta = Uniform(low: 0.008799999952316284, high: 0.013199999928474426),
	gamma = Uniform(low: 0.36480000615119934, high: 0.5472000241279602),
	delta = Uniform(low: 0.008799999952316284, high: 0.013199999928474426),
	alpha = Uniform(low: 0.4560000002384186, high: 0.6840000152587891),
	epsilon = Uniform(low: 0.13680000603199005, high: 0.20520000159740448),
	zeta = Uniform(low: 0.10000000149011612, high: 0.15000000596046448),
	XXlambdaXX = Uniform(low: 0.0272000003606081, high: 0.040800001472234726),
	eta = Uniform(low: 0.10000000149011612, high: 0.15000000596046448),
	rho = Uniform(low: 0.0272000003606081, high: 0.040800001472234726),
	theta = Uniform(low: 0.29679998755455017, high: 0.44519999623298645),
	kappa = Uniform(low: 0.01360000018030405, high: 0.020400000736117363),
	mu = Uniform(low: 0.01360000018030405, high: 0.020400000736117363),
	nu = Uniform(low: 0.

### Calibrate

In [23]:
num_iterations = 100
inferred_parameters = calibrate(ensemble, train_data, num_iterations=num_iterations, verbose=True, verbose_every=10)

iteration 0: loss = 10212.50804528594
iteration 10: loss = 9551.916983693838
iteration 20: loss = 7870.280524224043
iteration 30: loss = 7196.783444136381
iteration 40: loss = 6517.996141523123
iteration 50: loss = 6085.240189522505
iteration 60: loss = 5393.360059708357
iteration 70: loss = 4939.177564591169
iteration 80: loss = 4633.045360416174
iteration 90: loss = 4085.2261367738247


In [24]:
# Load saved parameters

save_path = "./saved_guides/"

with open(os.path.join(save_path, "inferred_parameters2.pkl"), "rb") as f:
    inferred_parameters = dill.load(f)
print("Loaded inferred parameters from file.")

Loaded inferred parameters from file.


In [26]:
num_samples = 3
ensemble_posterior_forecasts = sample(ensemble, all_timepoints, num_samples, inferred_parameters)

In [3]:
[*inferred_parameters.named_parameters()]

[('loc',
  Parameter containing:
  tensor([-0.3419, -1.8542, -0.9878, -1.0106,  1.2021,  1.2849,  2.2740, -1.5860,
           1.9740,  1.5441,  0.9700, -0.0286, -2.1124,  1.3206,  1.6324,  1.6865,
           0.2351,  0.1138,  0.2430,  0.1416, -0.1850, -0.0614, -0.3492,  0.0622,
          -0.1692,  0.8840,  0.0276, -0.5106, -0.1815, -0.0883,  0.0168, -0.1803,
           0.2660, -0.1332,  0.1727,  0.1170,  0.1163,  0.0087,  0.0054, -1.7854,
           1.9843], requires_grad=True)),
 ('scale_unconstrained',
  Parameter containing:
  tensor([ 0.3795, -0.5289,  0.3884, -1.6352, -1.0087,  0.3064, -0.0165, -0.0054,
           0.0226, -0.1191,  0.2943,  0.3943, -0.9570, -0.8787,  0.1069, -0.7997,
           0.4267,  0.4204,  0.4664,  0.4563,  0.4117,  0.4294,  0.4210,  0.4147,
           0.4345,  0.2505,  0.3819,  0.4337,  0.4461,  0.3777,  0.4550,  0.4119,
           0.4483,  0.4159,  0.4482,  0.4427,  0.4390,  0.3692,  0.4392, -2.2280,
          -0.0145], requires_grad=True))]